In [ ]:
import h3
import geopandas as gpd
from shapely import Point, Polygon, wkt
import matplotlib.pyplot as plt
import requests
import cartopy.crs as ccrs
import time

In [ ]:
## read in geometry of great plains outline
## downloaded from: https://cartographyvectors.com/map/1288-us-great-plains
usgp = gpd.read_file('us-great-plains_1288.geojson')
world = gpd.read_file('../day3_polygons/ne_10m_admin_0_countries.zip')
oceans = gpd.read_file('../day3_polygons/ne_10m_ocean.zip')

In [ ]:
# get geojson of great plains geometry
gj = usgp.loc[0,'geometry'].__geo_interface__

# identify hex names (at scale 3) that fill the geometry
h3_hexes = h3.polyfill_geojson(gj, 3)
print(f'there are {len(h3_hexes)} hexagons')

# construct geodataframe of hexagons
hexagons = gpd.GeoDataFrame(
    data={'hex':list(h3_hexes)},
    geometry=[Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in h3_hexes],
    crs=4326)

# and get centroids of each hexagon for querying elevation
centroids = gpd.GeoSeries([Point(h3.h3_to_geo(q)[::-1]) for q in h3_hexes], crs=4326)

def get_z(p):
    '''
    query ASTER GDEM for elevation data.
    api is limited to 1/second & 1000/day
    '''
    time.sleep(1)
    url = 'https://api.opentopodata.org/v1/aster30m'
    params={'locations':f'{p.y},{p.x}'}
    r = requests.get(url,params,timeout=2)
    return r.json()['results'][0]['elevation']

# get elevations and add to hexagons geodataframe
z = centroids.apply(lambda p: get_z(p))
hexagons['z'] = z

In [ ]:
# # export then read back in... so I don't have to download again
# hexagons.to_file('with_z.geojson', driver='GeoJSON')
# hxgn = gpd.read_file('with_z.geojson')

In [ ]:
# set projection
prj = ccrs.AlbersEqualArea(central_longitude=-96,
                           standard_parallels=[20,60],
                           globe=ccrs.Globe(datum='NAD83'))

fig,ax=plt.subplots(figsize=[8,8],
                    subplot_kw={'projection':prj},
                    edgecolor='k', linewidth=1)

world.to_crs(prj).plot(fc='none',
                       ec='none',
                       linewidth=0.8,
                       ax=ax)

oceans.to_crs(prj).plot(fc='#2f5e95',
                        ec='none',
                        ax=ax)

hexagons.to_crs(prj).plot(column='z',
                          cmap='gist_earth',
                          vmin=-2500,
                          vmax=2500, 
                          ec='w',
                          linewidth=0.5,
                          legend=False,
                          ax=ax)

ax.set_xlim(-2.5e6, 2.5e6)
ax.set_ylim(2e6, 7e6)

ax.set_title('         hexagons tile\n         the (great) plain(s)', font='DejaVu Sans Mono', loc='left', y=0.9)

ax.annotate(text='Data from: Natural Earth and ASTER GDEM (NASA/METI)',
            xy=(1,0.01), 
            xycoords='axes fraction', 
            ha='right',
            c='k', 
            backgroundcolor='w', 
            fontsize=8,
            font='DejaVu Sans Mono')

ax.annotate(text='by:tlohde',
            xy=(0.01,0.01), 
            xycoords='axes fraction',
            ha='left',
            c='w',
            fontsize=8,
            font='DejaVu Sans Mono')

ax.set_axis_off()

In [ ]:
fig.savefig('day9.png', bbox_inches='tight', dpi=300)